In [2]:
import numpy as np
from glob import glob


In [14]:
book_files = sorted(glob("/homes/80/kang/LOBS5/GOOG2017to2019/" + '*message*.npy'))
x=np.load(book_files[-5])
book_files[-5]

'/homes/80/kang/LOBS5/GOOG2017to2019/GOOG_2019-12-09_34200000_57600000_message_10_proc.npy'

In [17]:
x.shape

(522682, 14)

In [15]:
x[x[:,4]<500].shape

(522682, 14)

In [22]:
np.unique(x[:,-4:])

array([    -9999,      -257,      -250, ..., 999994678, 999995771,
       999996345])